In [1]:
!pwd

/panda/eedy/ion_los-angle/s1_d173/notebook


In [2]:
cp ../mintpy_split_ion_upsample/inputs/ionStack.h5 ../mintpy_split_ion_upsample/inputs/ifgramStack.h5 .

In [3]:
!modify_network.py ionStack.h5 --reset

ERROR 1: PROJ: proj_create_from_database: Open of /home/eedy/tools/mambaforge/envs/insar/share/proj failed
No lookup table (longitude or rangeCoord) found in files.
--------------------------------------------------
reset dataset 'dropIfgram' to True for all interferograms for file: ionStack.h5
All dropIfgram are already True, no need to reset.


In [4]:
!modify_network.py ionStack.h5 --reference ifgramStack.h5

ERROR 1: PROJ: proj_create_from_database: Open of /home/eedy/tools/mambaforge/envs/insar/share/proj failed
No lookup table (longitude or rangeCoord) found in files.
open ifgramStack file: ionStack.h5
number of interferograms: 837
--------------------------------------------------
use reference pairs info from file: ifgramStack.h5
number of interferograms in reference: 2731
date12 not in reference file: (2)
['20221210_20230115', '20230115_20230127']
--------------------------------------------------
number of interferograms to remove: 2
number of interferograms to keep  : 835
open file ionStack.h5 with r+ mode
update HDF5 dataset "/dropIfgram".
update MODIFICATION_TIME in HDF5 dataset "/unwrapPhase"
update MODIFICATION_TIME in HDF5 dataset "/coherence"
Done.


In [5]:
!modify_network.py ifgramStack.h5 --reset

ERROR 1: PROJ: proj_create_from_database: Open of /home/eedy/tools/mambaforge/envs/insar/share/proj failed
No lookup table (longitude or rangeCoord) found in files.
--------------------------------------------------
reset dataset 'dropIfgram' to True for all interferograms for file: ifgramStack.h5


In [6]:
!modify_network.py ifgramStack.h5  --reference ionStack.h5

ERROR 1: PROJ: proj_create_from_database: Open of /home/eedy/tools/mambaforge/envs/insar/share/proj failed
No lookup table (longitude or rangeCoord) found in files.
open ifgramStack file: ifgramStack.h5
number of interferograms: 2784
--------------------------------------------------
use reference pairs info from file: ionStack.h5
number of interferograms in reference: 835
date12 not in reference file: (1949)
['20150614_20150918', '20150614_20151012', '20150614_20161006', '20150614_20161030', '20150614_20170227', '20150614_20170603', '20150614_20170615', '20150801_20161006', '20150801_20161030', '20150801_20170227', '20150801_20170603', '20150801_20170615', '20150801_20170627', '20150801_20170709', '20150825_20161030', '20150825_20170227', '20150825_20170615', '20150825_20170627', '20150825_20170709', '20150825_20170721', '20150918_20170227', '20150918_20170615', '20150918_20170627', '20150918_20170709', '20150918_20170721', '20150918_20170802', '20151012_20161006', '20151012_20161030'

In [7]:
import h5py
import numpy as np

# 读取 ifgramStack.h5 文件
with h5py.File("ifgramStack.h5", "r+") as file:
    # 读取 "/dropIfgram" 数组和 "/unwrapPhase" 数组
    drop_ifgram = file["/dropIfgram"][:]
    unwrap_phase = file["/unwrapPhase"][:]
    date = file["/date"][:]
    bperp = file["/bperp"][:]

    # 确定要保留的数据的索引
    keep_indices = np.where(drop_ifgram == 1)[0]

    # 更新 "/unwrapPhase" 数组，只保留对应 dropIfgram 为 1 的数据
    unwrap_phase = unwrap_phase[keep_indices]
    date = date[keep_indices]
    drop_ifgram = drop_ifgram[keep_indices]
    bperp = bperp[keep_indices]

    # 删除对应 dropIfgram 为 0 的数据
    del file["/unwrapPhase"]
    file.create_dataset("/unwrapPhase", data=unwrap_phase)
    del file["/date"]
    file.create_dataset("/date", data=date)
    del file["/dropIfgram"]
    file.create_dataset("/dropIfgram", data=drop_ifgram)
    del file["/bperp"]
    file.create_dataset("/bperp", data=bperp)

    # 保存修改后的 ifgramStack.h5 文件
    file.flush()

# 读取 ionStack.h5 文件
with h5py.File("ionStack.h5", "r+") as file:
    # 读取 "/dropIfgram" 数组和 "/unwrapPhase" 数组
    drop_ifgram = file["/dropIfgram"][:]
    unwrap_phase = file["/unwrapPhase"][:]
    date = file["/date"][:]
    bperp = file["/bperp"][:]

    # 确定要保留的数据的索引
    keep_indices = np.where(drop_ifgram == 1)[0]

    # 更新 "/unwrapPhase" 数组，只保留对应 dropIfgram 为 1 的数据
    unwrap_phase = unwrap_phase[keep_indices]
    date = date[keep_indices]
    drop_ifgram = drop_ifgram[keep_indices]
    bperp = bperp[keep_indices]

    # 删除对应 dropIfgram 为 0 的数据
    del file["/unwrapPhase"]
    file.create_dataset("/unwrapPhase", data=unwrap_phase)
    del file["/date"]
    file.create_dataset("/date", data=date)
    del file["/dropIfgram"]
    file.create_dataset("/dropIfgram", data=drop_ifgram)
    del file["/bperp"]
    file.create_dataset("/bperp", data=bperp)

    # 保存修改后的 ionStack.h5 文件
    file.flush()

In [8]:
!diff.py ifgramStack.h5 ionStack.h5

ifgramStack.h5 - ['ionStack.h5'] --> ifgramStack_diff_ionStack.h5
the 1st input file is: ifgramStack
open ifgramStack file: ifgramStack.h5
open ifgramStack file: ionStack.h5
reading unwrapPhase from file ifgramStack.h5 ...
reading unwrapPhase from file ionStack.h5 ...
referencing to pixel (304,54) ...
delete exsited file: ifgramStack_diff_ionStack.h5
create HDF5 file: ifgramStack_diff_ionStack.h5 with w mode
create dataset /unwrapPhase of float32    in size of (835, 632, 226)      with compression=None
create dataset /bperp       of float32    in size of (835,)     with compression=None
create dataset /date        of |S8        in size of (835, 2)   with compression=None
create dataset /dropIfgram  of bool       in size of (835,)     with compression=None
finished writing to ifgramStack_diff_ionStack.h5
time used: 00 mins 32.3 secs


In [9]:
ionStack_path = 'ionStack.h5'
with h5py.File(ionStack_path, 'r') as file:
    date = file['date']
    date = date[()]
ion_date_list = np.char.decode(date)
ion_date_list = [f'{start_date}_{end_date}' for start_date, end_date in ion_date_list]

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from mintpy.view import prep_slice, plot_slice


for i in range(len(ion_date_list)):
    print(i)
    opt = f' --wrap -c cmy --noverbose --zero-mask'
    cmd_list = [f'view.py ifgramStack.h5 unwrap*{ion_date_list[i]} {opt} --title wrap ',
                f'view.py ionStack.h5 *{ion_date_list[i]} {opt} --title ion --ref-yx 548 59', 
                f'view.py ifgramStack_diff_ionStack.h5  *{ion_date_list[i]} {opt} --title correct']

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=[12, 8])
    for ax, cmd in zip(axs, cmd_list):
        data, atr, inps = prep_slice(cmd)
        plot_slice(ax, data, atr, inps)
    # 保存图片到pic_ion目录下，文件名为i
    plt.savefig(f'pic_ion/{ion_date_list[i]}.png')
    plt.show()
    plt.close()